# Load Data

In [2]:
from pathlib import Path
import json
import pandas as pd

path = Path('data')

In [ ]:
test_data = pd.read_csv(path/'mobile_ensemble_preds.csv')
test_data_lstm = test_data.filter(regex=(".*_lstm"))
# test_data_lstm['title'] = test_data['title']

test_data_trf= test_data.filter(regex=(".*_trf"))
# test_data_trf['title'] = test_data['title']
print(test_data_trf)

x_val = test_data['title']
y_val = test_data['Category']

# Ensemble Training

In [6]:
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
import numpy as np

Using TensorFlow backend.


In [7]:
from numpy import dstack

members = [test_data_lstm, test_data_trf]

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))
            
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
#     print(stackX)
    return stackX

In [15]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    
    # fit standalone model
    model = LogisticRegression()
    
    print(model)
    model.fit(stackedX, inputy)
    print(model)
    return model

In [ ]:
# fit stacked model using the ensemble
model = fit_stacked_model(members, x_val, y_val)

# Ensemble Evaluation

In [10]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
#     print(yhat)
    return yhat

In [ ]:
# evaluate model on test set
yhat = stacked_prediction(members, model, x_val)
acc = accuracy_score(y_val, yhat)

print('Stacked Test Accuracy: %.9f' % acc)

# Ensemble Prediction

In [ ]:
testset = pd.read_csv(path/"mobile_predictions.csv")


testset_lstm = testset.filter(regex=(".*_lstm"))
testset_trf= testset.filter(regex=(".*_trf"))

members_testset = [testset_lstm, testset_trf]

x_val=[]

for index,data in df.iterrows():
    x_val.append(data.tolist())
    
x_val = np.asarray(x_val)

preds = model.predict(x_val, verbose=1)

pred_cat = ([np.argmax(y, axis=None, out=None) + 31 for y in preds])
submission_mobile = pd.DataFrame({'Category':pred_cat})
print(submission_mobile.head())

submission_mobile.to_csv('submission_mobile.csv', index=False)

In [23]:
df = pd.concat([test_data_lstm,test_data_trf],axis=1)

x_val=[]

for index,data in df.iterrows():
    x_val.append(data.tolist())
    
x_val = np.asarray(x_val)

In [24]:
x_val.shape

(43941, 28)